## EDA Pass 04
#### Check what features are good predictors and consider narrowing features

In [255]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
import joblib

import seaborn as sns
SEED = 14

#### Create folder structure

In [256]:
import os
for d in ['orig\data', 'output']:
    os.makedirs(d, exist_ok=True)

### Use *_train for training and testing model
#### Build a baseline model
#### Use sklearns Pipelines

Import data to dataframe

In [257]:
df_accounts = pd.read_csv("orig/accounts_test.csv")
print(df_accounts.shape)
df_accounts.head(10)

(13575, 2)


,Customer,Account
0,802643959,Credit Card
1,802643959,Current
2,802643959,Investment
3,802643959,Loan
4,802643959,Deposit
5,742486575,Current
6,742486575,On Demand Deposit
7,742486575,Credit Card
8,753345746,Deposit
9,753345746,Credit Card


In [258]:
cols = df_accounts.Account.unique()


### Make each account a binary feature as each account customer relates to multiple accounts

In [259]:
df_accounts2 = pd.DataFrame(columns ={'Investment', 'Current', 'Credit_Card', 'On_Demand_Deposit','Mortgage', 'Joint', 'Deposit', 'Loan'})
df_accounts2

,Mortgage,Joint,On_Demand_Deposit,Deposit,Current,Credit_Card,Investment,Loan


In [260]:
frames = [df_accounts,df_accounts2]
df_accounts3 = pd.concat(frames)
df_accounts3

,Customer,Account,Mortgage,Joint,On_Demand_Deposit,Deposit,Current,Credit_Card,Investment,Loan
0,802643959.0,Credit Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,802643959.0,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,802643959.0,Investment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,802643959.0,Loan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,802643959.0,Deposit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
13570,723043620.0,Joint,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13571,723043620.0,Investment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13572,723043620.0,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13573,723043620.0,Credit Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [261]:
df_accounts3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13575 entries, 0 to 13574
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Customer           13575 non-null  float64
 1   Account            13575 non-null  object 
 2   Mortgage           0 non-null      object 
 3   Joint              0 non-null      object 
 4   On_Demand_Deposit  0 non-null      object 
 5   Deposit            0 non-null      object 
 6   Current            0 non-null      object 
 7   Credit_Card        0 non-null      object 
 8   Investment         0 non-null      object 
 9   Loan               0 non-null      object 
dtypes: float64(1), object(9)
memory usage: 1.1+ MB


In [262]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [263]:
df_accounts3['On_Demand_Deposit'].loc[df_accounts3['Account']=='On Demand Deposit'] = 1
df_accounts3['Credit_Card'].loc[df_accounts3['Account']=='Credit Card'] = 1
df_accounts3['Loan'].loc[df_accounts3['Account']=='Loan'] = 1
df_accounts3['Joint'].loc[df_accounts3['Account']=='Joint'] = 1
df_accounts3['Deposit'].loc[df_accounts3['Account']=='Deposit'] = 1
df_accounts3['Mortgage'].loc[df_accounts3['Account']=='Mortgage'] = 1
df_accounts3['Current'].loc[df_accounts3['Account']=='Current'] = 1
df_accounts3['Investment'].loc[df_accounts3['Account']=='Investment'] = 1



df_accounts3=df_accounts3.fillna(0)

In [264]:
df_accounts3 = df_accounts3.drop_duplicates()

In [265]:
df_accounts3_count = df_accounts3.groupby(['Customer']).count().reset_index()
df_accounts3_count = df_accounts3_count.rename(columns ={"Account": "num_accs"})
df_accounts4_count= df_accounts3_count[["Customer", "num_accs"]]
df_accounts4_count

,Customer,num_accs
0,708182832.0,4
1,708240660.0,5
2,708249778.0,4
3,708259785.0,3
4,708264579.0,6
...,...,...
3537,828151802.0,4
3538,828252238.0,3
3539,828255707.0,2
3540,828306544.0,2


In [266]:
df_accounts4 = (pd.merge(df_accounts4_count, df_accounts3, on='Customer'))
df_accounts4

,Customer,num_accs,Account,Mortgage,Joint,On_Demand_Deposit,Deposit,Current,Credit_Card,Investment,Loan
0,708182832.0,4,Deposit,0,0,0,1,0,0,0,0
1,708182832.0,4,Current,0,0,0,0,1,0,0,0
2,708182832.0,4,Credit Card,0,0,0,0,0,1,0,0
3,708182832.0,4,Investment,0,0,0,0,0,0,1,0
4,708240660.0,5,Investment,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
13476,828306544.0,2,Credit Card,0,0,0,0,0,1,0,0
13477,828306544.0,2,Current,0,0,0,0,1,0,0,0
13478,828331426.0,3,Deposit,0,0,0,1,0,0,0,0
13479,828331426.0,3,On Demand Deposit,0,0,1,0,0,0,0,0


In [267]:
df_accounts4 = df_accounts4.drop(columns='Account')


In [268]:
df_accounts4 =df_accounts4.groupby(['Customer']).sum().reset_index()

In [269]:
df_accounts4

,Customer,num_accs,Mortgage,Joint,On_Demand_Deposit,Deposit,Current,Credit_Card,Investment,Loan
0,708182832.0,16,0,0,0,1,1,1,1,0
1,708240660.0,25,0,0,0,1,1,1,1,1
2,708249778.0,16,1,0,0,0,1,1,1,0
3,708259785.0,9,0,0,0,1,1,1,0,0
4,708264579.0,36,0,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...
3537,828151802.0,16,0,0,1,0,1,1,1,0
3538,828252238.0,9,0,0,1,0,1,1,0,0
3539,828255707.0,4,0,0,0,0,1,1,0,0
3540,828306544.0,4,0,0,0,0,1,1,0,0


In [270]:
df_accounts4.Customer.astype(int)

0       708182832
1       708240660
2       708249778
3       708259785
4       708264579
          ...    
3537    828151802
3538    828252238
3539    828255707
3540    828306544
3541    828331426
Name: Customer, Length: 3542, dtype: int32

In [271]:
df_accounts4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3542 entries, 0 to 3541
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Customer           3542 non-null   float64
 1   num_accs           3542 non-null   int64  
 2   Mortgage           3542 non-null   int64  
 3   Joint              3542 non-null   int64  
 4   On_Demand_Deposit  3542 non-null   int64  
 5   Deposit            3542 non-null   int64  
 6   Current            3542 non-null   int64  
 7   Credit_Card        3542 non-null   int64  
 8   Investment         3542 non-null   int64  
 9   Loan               3542 non-null   int64  
dtypes: float64(1), int64(9)
memory usage: 276.8 KB


In [272]:
df_customers = pd.read_csv("orig/customers_test.csv")
print(df_customers.shape)
df_customers.head(10)

(3542, 14)


,Customer,Card,Start_Date,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income
0,774663629,Gold,2020-01-01,2,29663.0,1743,27920.0,0.059,51,M,Graduate,Single,3,€50K - €80K
1,720420396,Silver,2020-11-01,2,2032.0,1195,837.0,0.588,45,F,Unknown,Divorced,3,€30K - €50K
2,815283379,Silver,2020-01-01,1,17268.0,1197,16071.0,0.069,57,M,Graduate,Single,3,€100K +
3,764861610,Silver,2020-05-01,3,2264.0,0,2264.0,0.000,38,F,Primary level,Single,2,Less than €30K
4,742798818,Silver,2020-01-01,1,2192.0,1464,728.0,0.668,45,F,MSc,Unknown,4,Less than €30K
5,763383343,Silver,2019-10-01,3,1613.0,0,1613.0,0.000,45,M,Graduate,Single,2,€80K - €100K
6,750463896,Silver,2020-08-01,2,2818.0,1656,1162.0,0.588,38,M,Second level,Married,2,€100K +
7,786288890,Platinum,2020-01-01,3,34516.0,1864,32652.0,0.054,58,F,Second level,Divorced,2,Unknown
8,735694396,Silver,2020-01-01,0,1438.3,0,1438.3,0.000,45,F,Unknown,Single,3,Less than €30K
9,745193792,Silver,2020-02-01,4,2482.0,916,1566.0,0.369,53,F,Post LC,Single,1,Less than €30K


In [273]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3542 entries, 0 to 3541
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer                   3542 non-null   int64  
 1   Card                       3542 non-null   object 
 2   Start_Date                 3542 non-null   object 
 3   Customer_Service_Calls     3542 non-null   int64  
 4   Credit_Limit               3542 non-null   float64
 5   Total_Revolving_Balance    3542 non-null   int64  
 6   Average_Open_To_Buy        3542 non-null   float64
 7   Average_Utilisation_Ratio  3542 non-null   float64
 8   Age                        3542 non-null   int64  
 9   Gender                     3542 non-null   object 
 10  Education                  3542 non-null   object 
 11  Marital_Status             3542 non-null   object 
 12  Dependents                 3542 non-null   int64  
 13  Income                     3542 non-null   objec

What % of customers churn?

For unknown age set the age to the median and record where age was not set

#### Create a feature based on start date that will be the number of months the account/customer has been with the bank

In [274]:
df_customers.Start_Date = pd.to_datetime(df_customers.Start_Date)
df_customers['nb_months'] = ((pd.to_datetime("today") - df_customers.Start_Date)/np.timedelta64(1, 'M'))
df_customers['nb_months'] = df_customers['nb_months'].astype(int)
df_customers

,Customer,Card,Start_Date,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income,nb_months
0,774663629,Gold,2020-01-01,2,29663.0,1743,27920.0,0.059,51,M,Graduate,Single,3,€50K - €80K,38
1,720420396,Silver,2020-11-01,2,2032.0,1195,837.0,0.588,45,F,Unknown,Divorced,3,€30K - €50K,28
2,815283379,Silver,2020-01-01,1,17268.0,1197,16071.0,0.069,57,M,Graduate,Single,3,€100K +,38
3,764861610,Silver,2020-05-01,3,2264.0,0,2264.0,0.000,38,F,Primary level,Single,2,Less than €30K,34
4,742798818,Silver,2020-01-01,1,2192.0,1464,728.0,0.668,45,F,MSc,Unknown,4,Less than €30K,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,784491579,Silver,2020-07-01,3,2485.0,1648,837.0,0.663,49,F,Graduate,Divorced,4,€30K - €50K,32
3538,750122800,Silver,2019-07-01,4,19075.0,0,19075.0,0.000,48,M,Graduate,Single,3,€50K - €80K,45
3539,777178751,Silver,2020-10-01,3,2229.0,1791,438.0,0.803,37,F,Primary level,Married,3,Less than €30K,29
3540,711974616,Silver,2020-01-01,2,10273.0,1657,8616.0,0.161,52,F,Unknown,Single,3,Unknown,38


In [275]:
df_customers.insert(loc=10, column = 'Age_was_NA',value=0)

In [276]:
df_customers['Age_was_NA'].loc[df_customers['Age'].isnull()] = 1

In [277]:
median_age = df_customers.Age.median()
median_age

46.0

In [278]:
df_customers['Age'].loc[df_customers['Age'].isnull()] = median_age

In [279]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3542 entries, 0 to 3541
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Customer                   3542 non-null   int64         
 1   Card                       3542 non-null   object        
 2   Start_Date                 3542 non-null   datetime64[ns]
 3   Customer_Service_Calls     3542 non-null   int64         
 4   Credit_Limit               3542 non-null   float64       
 5   Total_Revolving_Balance    3542 non-null   int64         
 6   Average_Open_To_Buy        3542 non-null   float64       
 7   Average_Utilisation_Ratio  3542 non-null   float64       
 8   Age                        3542 non-null   int64         
 9   Gender                     3542 non-null   object        
 10  Age_was_NA                 3542 non-null   int64         
 11  Education                  3542 non-null   object        
 12  Marita

In [280]:
df_transactions = pd.read_csv("orig/transactions_test.csv")
print(df_transactions.shape)
df_transactions.head(5)

(227447, 3)


,Customer,Date,Amount
0,802643959,2022-01-03,37.22
1,802643959,2022-01-15,83.26
2,802643959,2022-02-25,9.76
3,802643959,2022-01-26,10.33
4,802643959,2022-01-06,10.88


In [281]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227447 entries, 0 to 227446
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Customer  227447 non-null  int64  
 1   Date      227447 non-null  object 
 2   Amount    227447 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 5.2+ MB


### Aggregate the transaction per customer initially

In [282]:
df_transactions_count = df_transactions.groupby(['Customer']).count().reset_index()
df_transactions_count = df_transactions_count.rename(columns ={"Date": "num_trans"})
df_transactions_count= df_transactions_count.drop(columns =["Amount"])
df_transactions_count

,Customer,num_trans
0,708182832,64
1,708240660,41
2,708249778,60
3,708259785,121
4,708264579,36
...,...,...
3537,828151802,69
3538,828252238,90
3539,828255707,85
3540,828306544,85


In [283]:
df_transactions = df_transactions.groupby(['Customer']).sum(['Amount']).reset_index()
df_transactions

,Customer,Amount
0,708182832,4341.01
1,708240660,2349.01
2,708249778,2120.98
3,708259785,15195.00
4,708264579,1302.02
...,...,...
3537,828151802,5290.00
3538,828252238,4290.02
3539,828255707,4657.99
3540,828306544,8186.00


In [284]:
df_transactions2 = (pd.merge(df_transactions_count, df_transactions, on='Customer'))
df_transactions2

,Customer,num_trans,Amount
0,708182832,64,4341.01
1,708240660,41,2349.01
2,708249778,60,2120.98
3,708259785,121,15195.00
4,708264579,36,1302.02
...,...,...,...
3537,828151802,69,5290.00
3538,828252238,90,4290.02
3539,828255707,85,4657.99
3540,828306544,85,8186.00


### Identify Categorical and numerical features
Looking at the customer df only initially

for null marital status set to unkown and record where it was previously null


In [285]:
df_customers.insert(loc=14, column = 'M_Stat_was_NA',value=0)

In [286]:
df_customers['M_Stat_was_NA'].loc[df_customers['Marital_Status'].isnull()] = 1

In [287]:
df_customers['Marital_Status'].loc[df_customers['Marital_Status'].isnull()] = 'Unknown'

In [288]:
df_customers.Gender = pd.Categorical(df_customers.Gender)
df_customers.Marital_Status = pd.Categorical(df_customers.Marital_Status)
df_customers.Card = pd.Categorical(df_customers.Card)
df_customers.Income = pd.Categorical(df_customers.Income)

In [289]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3542 entries, 0 to 3541
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Customer                   3542 non-null   int64         
 1   Card                       3542 non-null   category      
 2   Start_Date                 3542 non-null   datetime64[ns]
 3   Customer_Service_Calls     3542 non-null   int64         
 4   Credit_Limit               3542 non-null   float64       
 5   Total_Revolving_Balance    3542 non-null   int64         
 6   Average_Open_To_Buy        3542 non-null   float64       
 7   Average_Utilisation_Ratio  3542 non-null   float64       
 8   Age                        3542 non-null   int64         
 9   Gender                     3542 non-null   category      
 10  Age_was_NA                 3542 non-null   int64         
 11  Education                  3542 non-null   object        
 12  Marita

### Merge the customer and accounts dataframes

In [290]:
df_cust_acc = (pd.merge(df_customers, df_accounts4, on='Customer'))

In [291]:
df_cust_acc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3542 entries, 0 to 3541
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Customer                   3542 non-null   int64         
 1   Card                       3542 non-null   category      
 2   Start_Date                 3542 non-null   datetime64[ns]
 3   Customer_Service_Calls     3542 non-null   int64         
 4   Credit_Limit               3542 non-null   float64       
 5   Total_Revolving_Balance    3542 non-null   int64         
 6   Average_Open_To_Buy        3542 non-null   float64       
 7   Average_Utilisation_Ratio  3542 non-null   float64       
 8   Age                        3542 non-null   int64         
 9   Gender                     3542 non-null   category      
 10  Age_was_NA                 3542 non-null   int64         
 11  Education                  3542 non-null   object        
 12  Marita

In [292]:
df_cust_acc.On_Demand_Deposit = pd.Categorical(df_cust_acc.On_Demand_Deposit)
df_cust_acc.Investment = pd.Categorical(df_cust_acc.Investment)
df_cust_acc.Credit_Card = pd.Categorical(df_cust_acc.Credit_Card)
df_cust_acc.Loan = pd.Categorical(df_cust_acc.Loan)
df_cust_acc.Joint = pd.Categorical(df_cust_acc.Joint)
df_cust_acc.Mortgage = pd.Categorical(df_cust_acc.Mortgage)
df_cust_acc.Current = pd.Categorical(df_cust_acc.Current)
df_cust_acc.Deposit = pd.Categorical(df_cust_acc.Deposit)

In [293]:
df_cust_acc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3542 entries, 0 to 3541
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Customer                   3542 non-null   int64         
 1   Card                       3542 non-null   category      
 2   Start_Date                 3542 non-null   datetime64[ns]
 3   Customer_Service_Calls     3542 non-null   int64         
 4   Credit_Limit               3542 non-null   float64       
 5   Total_Revolving_Balance    3542 non-null   int64         
 6   Average_Open_To_Buy        3542 non-null   float64       
 7   Average_Utilisation_Ratio  3542 non-null   float64       
 8   Age                        3542 non-null   int64         
 9   Gender                     3542 non-null   category      
 10  Age_was_NA                 3542 non-null   int64         
 11  Education                  3542 non-null   object        
 12  Marita

In [294]:
df_cust_acc_tran = (pd.merge(df_cust_acc, df_transactions2, on='Customer'))

In [295]:
df_cust_acc_tran.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3542 entries, 0 to 3541
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Customer                   3542 non-null   int64         
 1   Card                       3542 non-null   category      
 2   Start_Date                 3542 non-null   datetime64[ns]
 3   Customer_Service_Calls     3542 non-null   int64         
 4   Credit_Limit               3542 non-null   float64       
 5   Total_Revolving_Balance    3542 non-null   int64         
 6   Average_Open_To_Buy        3542 non-null   float64       
 7   Average_Utilisation_Ratio  3542 non-null   float64       
 8   Age                        3542 non-null   int64         
 9   Gender                     3542 non-null   category      
 10  Age_was_NA                 3542 non-null   int64         
 11  Education                  3542 non-null   object        
 12  Marita

In [298]:
categorical_features = ['Gender', 'Marital_Status', 'On_Demand_Deposit', 'Investment','Loan', 'Joint', 'Mortgage', 'Current', 'Deposit', 'Card','Income', 'Education']
#categorical_features = ['Gender', 'Marital_Status', 'Card','Income', 'Credit_Card', 'Mortgage']
numerical_features = ['Customer_Service_Calls','Total_Revolving_Balance', 'Average_Utilisation_Ratio','Amount', 'num_trans', 'num_accs', 'nb_months']


#other_features = ['Education']


features = categorical_features + numerical_features #+ other_features



variables = features 

2nd pass we will still drop missing values for now

In [299]:
df_model = df_cust_acc_tran[variables].dropna()
df_model.shape
df_model

,Gender,Marital_Status,On_Demand_Deposit,Investment,Loan,Joint,Mortgage,Current,Deposit,Card,Income,Education,Customer_Service_Calls,Total_Revolving_Balance,Average_Utilisation_Ratio,Amount,num_trans,num_accs,nb_months
0,M,Single,0,0,0,0,0,1,1,Gold,€50K - €80K,Graduate,2,1743,0.059,14638.00,93,9,38
1,F,Divorced,0,0,0,0,0,0,0,Silver,€30K - €50K,Unknown,2,1195,0.588,4478.97,80,1,28
2,M,Single,0,0,0,0,0,1,0,Silver,€100K +,Graduate,1,1197,0.069,4077.01,90,4,38
3,F,Single,0,1,1,0,0,1,1,Silver,Less than €30K,Primary level,3,0,0.000,1204.02,28,25,34
4,F,Unknown,1,0,1,1,1,0,1,Silver,Less than €30K,MSc,1,1464,0.668,4905.03,89,36,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,F,Divorced,0,0,0,0,0,0,0,Silver,€30K - €50K,Graduate,3,1648,0.663,4208.03,71,1,32
3538,M,Single,1,0,0,0,0,1,0,Silver,€50K - €80K,Graduate,4,0,0.000,2013.01,32,9,45
3539,F,Married,1,1,0,0,1,1,1,Silver,Less than €30K,Primary level,3,1791,0.803,1582.98,32,36,29
3540,F,Single,0,0,0,0,0,1,1,Silver,Unknown,Unknown,2,1657,0.161,3777.94,70,9,38


In [300]:
from sklearn.preprocessing import  StandardScaler

In [301]:
scaler = StandardScaler()

In [302]:
df_model[numerical_features] = scaler.fit_transform(df_model[numerical_features])

In [303]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier

### Build a Pipeline

In [304]:
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
column_transformer = ColumnTransformer([
    # List of transformation to apply and which columns to apply it to. Each entry here has structure 
    #( NAME,                  CODE,                           COLUMNS_TO_APPLY),
    ( 'categorical_features', preprocessing.OneHotEncoder(),  categorical_features ),
    ( 'numerical_features',   preprocessing.StandardScaler(), numerical_features ),
])

In [305]:
filename = "Completed_model_01.joblib"

In [306]:
loaded_model = joblib.load(filename)

In [307]:
df_cust_acc_tran["Churn"] = loaded_model.predict(df_model)

In [308]:
df_cust_acc_tran

,Customer,Card,Start_Date,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,...,Joint,On_Demand_Deposit,Deposit,Current,Credit_Card,Investment,Loan,num_trans,Amount,Churn
0,774663629,Gold,2020-01-01,2,29663.0,1743,27920.0,0.059,51,M,...,0,0,1,1,1,0,0,93,14638.00,0
1,720420396,Silver,2020-11-01,2,2032.0,1195,837.0,0.588,45,F,...,0,0,0,0,1,0,0,80,4478.97,0
2,815283379,Silver,2020-01-01,1,17268.0,1197,16071.0,0.069,57,M,...,0,0,0,1,1,0,0,90,4077.01,0
3,764861610,Silver,2020-05-01,3,2264.0,0,2264.0,0.000,38,F,...,0,0,1,1,1,1,1,28,1204.02,0
4,742798818,Silver,2020-01-01,1,2192.0,1464,728.0,0.668,45,F,...,1,1,1,0,1,0,1,89,4905.03,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,784491579,Silver,2020-07-01,3,2485.0,1648,837.0,0.663,49,F,...,0,0,0,0,1,0,0,71,4208.03,0
3538,750122800,Silver,2019-07-01,4,19075.0,0,19075.0,0.000,48,M,...,0,1,0,1,1,0,0,32,2013.01,1
3539,777178751,Silver,2020-10-01,3,2229.0,1791,438.0,0.803,37,F,...,0,1,1,1,1,1,0,32,1582.98,0
3540,711974616,Silver,2020-01-01,2,10273.0,1657,8616.0,0.161,52,F,...,0,0,1,1,1,0,0,70,3777.94,0


In [309]:
df_cust_acc_tran.reset_index()

,index,Customer,Card,Start_Date,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,...,Joint,On_Demand_Deposit,Deposit,Current,Credit_Card,Investment,Loan,num_trans,Amount,Churn
0,0,774663629,Gold,2020-01-01,2,29663.0,1743,27920.0,0.059,51,...,0,0,1,1,1,0,0,93,14638.00,0
1,1,720420396,Silver,2020-11-01,2,2032.0,1195,837.0,0.588,45,...,0,0,0,0,1,0,0,80,4478.97,0
2,2,815283379,Silver,2020-01-01,1,17268.0,1197,16071.0,0.069,57,...,0,0,0,1,1,0,0,90,4077.01,0
3,3,764861610,Silver,2020-05-01,3,2264.0,0,2264.0,0.000,38,...,0,0,1,1,1,1,1,28,1204.02,0
4,4,742798818,Silver,2020-01-01,1,2192.0,1464,728.0,0.668,45,...,1,1,1,0,1,0,1,89,4905.03,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,3537,784491579,Silver,2020-07-01,3,2485.0,1648,837.0,0.663,49,...,0,0,0,0,1,0,0,71,4208.03,0
3538,3538,750122800,Silver,2019-07-01,4,19075.0,0,19075.0,0.000,48,...,0,1,0,1,1,0,0,32,2013.01,1
3539,3539,777178751,Silver,2020-10-01,3,2229.0,1791,438.0,0.803,37,...,0,1,1,1,1,1,0,32,1582.98,0
3540,3540,711974616,Silver,2020-01-01,2,10273.0,1657,8616.0,0.161,52,...,0,0,1,1,1,0,0,70,3777.94,0


In [312]:
df_cust_acc_tran['Churn'].loc[df_cust_acc_tran['Churn']==1] = 'Yes'
df_cust_acc_tran['Churn'].loc[df_cust_acc_tran['Churn']==0] = 'No'

In [313]:
results = df_cust_acc_tran[["Customer", "Churn"]]

In [314]:
results.to_csv('submission.csv', index=False)

In [85]:
##Submitted 03/15/2023

In [94]:
results.groupby(['Customer']).size()

Customer
708182832    1
708240660    1
708249778    1
708259785    1
708264579    1
            ..
828151802    1
828252238    1
828255707    1
828306544    1
828331426    1
Length: 3542, dtype: int64